# multimethod

In [14]:
(defmulti say-my-possible-status 
    (fn [myself] (:age myself)))
(defmethod say-my-possible-status 
    10 [myself] 
    (println (str (:name myself) " possibly know nothing about anything")))
(defmethod say-my-possible-status 
    30 [myself] 
    (println (str (:name myself) " possibly is trying to find out why he is so confused")))
(defmethod say-my-possible-status 
    :default [myself] 
    (println (str (:name myself) " is in a status of mystery at age of " (:age myself))))
(say-my-possible-status {:age 10 :name :woosley})
(say-my-possible-status {:age 30 :name :woosley})
(say-my-possible-status {:age 50 :name :woosley})


:woosley possibly know nothing about anything
:woosley possibly is trying to find out why he is so confused
:woosley is in a status of mystery at age of 50


nil #object[clojure.lang.MultiFn 0x633d8869 "clojure.lang.MultiFn@633d8869"] #object[clojure.lang.MultiFn 0x633d8869 "clojure.lang.MultiFn@633d8869"] #object[clojure.lang.MultiFn 0x633d8869 "clojure.lang.MultiFn@633d8869"] nil nil nil

multimethod可以在不同的namespace之间进行扩展

In [13]:
*ns*
(ns foo)
(defmulti test-nil (fn [data] (nil? data)))
(ns user)
(defmethod foo/test-nil true [data] (println "data is nil"))
(foo/test-nil nil)

data is nil


#object[clojure.lang.Namespace 0x2fcd565d "user"] nil nil nil #object[clojure.lang.MultiFn 0x29e0fc62 "clojure.lang.MultiFn@29e0fc62"] nil

# protocol

protocol定义扩展一种数据类型的协议。它通过**defprotocol**定义扩展类型需要实现的函数，让后通过**extend-type TYPE**来实现具体的扩展细节。

In [8]:
(defprotocol livehard
    (think [x] "Think hard and live hard")
    (sing [x] [x y] "Sing to relieve"))
(extend-type java.lang.String livehard
    (think [x] "You see me when I am in a rock status")
    (sing ([x] "Start, start, I am gonna explod")
          ([x y] "Continued, the red eye" )))
          
(think "woosley")
(sing "song")
(sing "good" "song")

livehard nil "You see me when I am in a rock status" "Start, start, I am gonna explod" "Continued, the red eye"

其中注意 
- `sing [x] [x y]` 表示sing接受两组不同形式的参数
- 第一个参数必须保证类型和扩展的类型匹配，其他参数则没有要求
- 可以通过扩展`java.lang.Object`来为所有的类型扩展默认协议。
- 协议中的原型定义不能使用**[x & other-args]**形式

当我们需要对多种类型进行协议扩展的时候，可以使用**extend-protocol**形式

In [10]:
(extend-protocol livehard
java.lang.String 
(think [x] "You see me when I am in a rock status")
(sing ([x] "Start, start, I am gonna explod")
      ([x y] "Continued, the red eye" ))
java.lang.Object
(think [x] "Starting from the basic, I am here")
(sing ([x] "You see this is the fundemental stuff")
      ([x y] "show me your big ticket")))
      
(sing 19)

nil "You see this is the fundemental stuff"

# record

record的行为和map很相似，使用`defrecord`来定义一个record

In [1]:
(defrecord human [name age gender])

user.human

当一个record被定义之后，会自动生成几个不同的初始化record的方法

- recodeName.
- ->recodeName
- map->recodeName

In [2]:
(human. :woosley 30 :male)
(->human :woosley 30 :male)
(map->human {:name :woosley :age 30 :gender :male})

#user.human{:name :woosley, :age 30, :gender :male} #user.human{:name :woosley, :age 30, :gender :male} #user.human{:name :woosley, :age 30, :gender :male}

recode可以在别的名字空间通过**:import**导入

In [4]:
(ns foo
(:import [user human]))
(def me (human. :woosley 30 :male))
(.name me)
(:age me)
(get me :gender)
(ns user)

nil #'foo/me :woosley 30 :male nil

record可以使用所有map的方法，值得注意的是dissoc返回值不再是一个record，而是map

In [9]:
(def me (human. :woosely :30 :male))
(assoc me :age 31)
(dissoc me :age)

#'user/me #user.human{:name :woosely, :age 31, :gender :male} {:name :woosely, :gender :male}

record可以直接用来扩展protocol

In [11]:
(defprotocol behavior 
    (getting-old [x] "make sure that you get old"))
(defrecord human [name age gender]
behavior 
(getting-old [x] (str (:name x) " You are now " (+ 1 (:age x)))))
(def me (human. :woosley 30 :male))
(getting-old me)

behavior user.human #'user/me ":woosley You are now 31"